In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Q1.  Who are the top 5 users in Sydney with the highest number of followers? List their login in order, comma-separated

In [5]:
users = pd.read_csv('users.csv')
users.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,nicknochnack,Nicholas Renotte,NaN,"Sydney, Australia",NaN,NaN,Data Science guy @IBM,193,11682,5,2013-11-15T15:06:35Z
1,brendangregg,Brendan Gregg,INTEL,"Sydney, Australia",NaN,NaN,Cloud computing performance engineer.,42,8022,0,2011-10-04T08:48:56Z
2,cornflourblue,Jason Watmore,POINT BLANK DEVELOPMENT,Sydney Australia,NaN,NaN,"Web Developer and Blogger, currently focusing ...",187,6701,0,2011-05-13T04:38:16Z
3,0vm,Ak,NaN,"Sydney, Australia",NaN,True,erm hai,10,6567,3,2021-03-02T06:55:10Z
4,davecheney,Dave Cheney,GITHUB,"Sydney, Australia",NaN,NaN,I service society by rocking.,163,6557,42,2008-04-14T04:00:20Z


In [6]:
users['hireable'] = users['hireable'].fillna(False).astype(bool)

C:\Users\ajj\AppData\Local\Temp\ipykernel_28636\1926759118.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users['hireable'] = users['hireable'].fillna(False).astype(bool)


In [7]:
top5 = users.sort_values(by='followers', ascending=False).head()
print(','.join(top5['login'].tolist()))

nicknochnack,brendangregg,cornflourblue,0vm,davecheney


#### Q2. Who are the 5 earliest registered GitHub users in Sydney? List their login in ascending order of created_at, comma-separated.

In [8]:
users['created_at'] = pd.to_datetime(users['created_at'])

In [9]:
top_earliest = users.sort_values(by='created_at').head()
print(','.join(top_earliest['login'].tolist()))

dylanegan,cjheath,freshtonic,dhowden,mikel


#### Q3. What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [10]:
repos = pd.read_csv('repositories.csv')
repos.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,nicknochnack,nicknochnack/ActionDetectionforSignLanguage,2021-06-16T09:12:01Z,408,408,Jupyter Notebook,True,True,NaN
1,nicknochnack,nicknochnack/AdvancedHandPoseWithMediaPipe,2021-04-25T00:13:57Z,37,37,Jupyter Notebook,True,True,NaN
2,nicknochnack,nicknochnack/ANPRwithPython,2020-12-13T04:39:49Z,106,106,Jupyter Notebook,True,True,NaN
3,nicknochnack,nicknochnack/AutoStreamlit,2022-10-25T21:25:41Z,72,72,Jupyter Notebook,True,True,NaN
4,nicknochnack,nicknochnack/beeagent,2024-09-19T08:05:08Z,8,8,TypeScript,True,True,NaN


In [11]:
repos['license_name'].value_counts().head(3)

license_name
mit           5961
other         1780
apache-2.0    1733
Name: count, dtype: int64

#### Q4. Which company do the majority of these developers work at?

In [9]:
users['company'].value_counts().head(1)

company
ADOBE    20
Name: count, dtype: int64

#### Q5. Which programming language is most popular among these users?

In [10]:
repos['language'].value_counts().head(1)

language
JavaScript    1395
Name: count, dtype: int64

#### Q6. Which programming language is the second most popular among users who joined after 2020?

In [11]:
users_after_2020 = users[users['created_at'] > '2020-01-01']
users_after_2020.head()

,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
85,markh0rr,NaN,STACKSYNC FULL STACK DEVELOPER | EPFL CS STUDENT,Basel | Lausanne Switzerland,NaN,False,MAKE SOMETHING PEOPLE WANT.,4,41,77,2021-02-11 11:10:18+00:00
94,leonden,Leon,HOME,Basel,info@leonhochwimmer.com,False,🪐🫧🦋,35,36,95,2022-01-17 09:06:41+00:00
147,lev1nn,Levin,COOP GENOSSENSCHAFT,"Basel, Switzerland",levin.schaller@bbzbl-it.ch,False,NaN,3,25,28,2022-04-19 09:04:34+00:00
155,yalais,Yannic Lais,NaN,Basel,NaN,False,NaN,4,24,30,2021-10-01 06:13:00+00:00
162,revitron,Revitron,NaN,Basel,NaN,False,A Python wrapper for the Revit API based on th...,15,23,0,2020-05-16 15:22:18+00:00


In [12]:
repos_2020 = repos[repos['login'].isin(users_after_2020['login'].tolist())]
repos_2020['language'].value_counts().head()

language
Python        35
PHP           26
HTML          23
Java          22
JavaScript    16
Name: count, dtype: int64

#### Q7. Which language has the highest average number of stars per repository? 

In [13]:
avg_stars = repos.groupby('language')['stargazers_count'].mean()
top_lang = avg_stars.idxmax()
top_stars = avg_stars.max()
print(top_lang, top_stars)

PureScript 114.0


#### Q8. Let's define leader_strength as followers / (1 + following). Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [14]:
users['leader_strength'] = users['followers'] / (1 + users['following'])
top5_lead = users.sort_values(by='leader_strength', ascending=False).head()
print(','.join(top5_lead['login'].tolist()))

dpryan79,wasserth,ravage84,elanmart,quadbiolab


#### Q9. What is the correlation between the number of followers and the number of public repositories among users in Basel?

In [15]:
correlation = users['followers'].corr(users['public_repos'])
correlation

0.34406396712642345

#### Q10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.

In [16]:
import csv
followers = []
public_repos = []
with open('users.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        followers_count = int(row['followers'])
        public_repos_count = int(row['public_repos'])
        followers.append(followers_count)
        public_repos.append(public_repos_count)
if len(followers) > 1 and len(public_repos) > 1:
    slope, intercept = np.polyfit(public_repos, followers, 1)
    
    print(f"{slope:.3f}")
else:
    print("Error")

0.672


#### Q11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?

In [17]:
if repos['has_projects'].dtype == 'object':
    repos['has_projects'] = repos['has_projects'].map({'true': True, 'false': False})
if repos['has_wiki'].dtype == 'object':
    repos['has_wiki'] = repos['has_wiki'].map({'true': True, 'false': False})
    
correlation = repos['has_projects'].corr(repos['has_wiki'])
    
print(round(correlation, 3))

0.312


#### Q12. Do hireable users follow more people than those who are not hireable?

In [18]:
hireable_avg_following = users[users['hireable'] == True]['following'].mean()
non_hireable_avg_following = users[users['hireable'] == False]['following'].mean()
difference = hireable_avg_following - non_hireable_avg_following
difference

46.94048144876325

#### Q13. Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode characters) with followers? (Ignore people without bios)

In [19]:
from sklearn.linear_model import LinearRegression
users_with_bio = users[(users['bio'].notna()) & (users['bio'] != '')].copy()
users_with_bio.loc[:, 'bio_len'] = users_with_bio['bio'].str.len()

X = users_with_bio['bio_len'].values.reshape(-1,1)
y = users_with_bio['followers']

lr2 = LinearRegression()
lr2.fit(X, y)
lr2.coef_[0]

0.37596931225094793

#### Q14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

In [20]:
import csv
from collections import Counter
from datetime import datetime

weekend_repo_counts = Counter()

with open('repositories.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    
    for row in reader:
        created_at = row.get('created_at', '')
        if created_at:
            created_date = datetime.fromisoformat(created_at[:-1])  
            
            if created_date.weekday() in [5, 6]:
                user_login = row['login']
                weekend_repo_counts[user_login] += 1  

top_users = weekend_repo_counts.most_common(5)

top_logins = [user[0] for user in top_users]

print(','.join(top_logins))

marcossegovia,tbreuss,ioolkos,BaselHack,maysam


#### Q15. Do people who are hireable share their email addresses more often?

In [21]:
fraction_hierable = users[users['hireable'] == True]['email'].notna().mean()
fraction_non_hierable = users[users['hireable'] == False]['email'].notna().mean()
diff = fraction_hierable - fraction_non_hierable
diff

0.05140238515901058

#### Q16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)

In [22]:
new_users = users[users['name'].notna()].copy()
new_users['surname'] = new_users['name'].str.split().str[-1].str.strip()
surname_counts = new_users['surname'].value_counts()
max_count = surname_counts.max()
common_surnames = surname_counts[surname_counts == max_count].index.tolist()
common_surnames.sort()
print(','.join(common_surnames))

Arnold,Brand,Christensen,Fink,GmbH,Group,Guggisberg,Landolt,Roth,Tan


In [23]:
# users = users.fillna("").astype(str)
# repos = repos.fillna("").astype(str)

# repos['has_wiki'] = repos['has_wiki'].astype(str).replace({'True': 'true', 'False': 'false'})
# repos['has_projects'] = repos['has_projects'].astype(str).replace({'True': 'true', 'False': 'false'})

# # users.to_csv('users1.csv', index=False)
# repos.to_csv('repos1.csv', index=False)